<a href="https://colab.research.google.com/github/choki0715/lecture/blob/master/quantization_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Model Quantization Techniques Comparison

이 노트북에서는 다양한 양자화 기법들을 비교합니다:
- **GPTQ** (Generalized Post-Training Quantization)
- **AWQ** (Activation-aware Weight Quantization)
- **BitsAndBytes** (8-bit and 4-bit quantization)
- **Dynamic Quantization** (PyTorch native)

## 📦 1. 라이브러리 설치

In [1]:
# 기본 라이브러리
!pip install -q transformers accelerate

# GPTQ
!pip install -q auto-gptq optimum

# AWQ
!pip install -q autoawq

# BitsAndBytes (8-bit, 4-bit)
!pip install -q bitsandbytes

print("✅ 모든 라이브러리 설치 완료!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.1/126.1 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 4.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.1 MB/s et

In [2]:
# CUDA 버전 확인
!nvcc --version

# CUDA 11.8용 (Colab T4/L4)
!pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.8/autoawq-0.1.8+cu118-cp310-cp310-linux_x86_64.whl

# 또는 CUDA 12.1용
!pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.8/autoawq-0.1.8+cu121-cp310-cp310-linux_x86_64.whl

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
ERROR: autoawq-0.1.8+cu118-cp310-cp310-linux_x86_64.whl is not a supported wheel on this platform.
ERROR: autoawq-0.1.8+cu121-cp310-cp310-linux_x86_64.whl is not a supported wheel on this platform.


In [ ]:
import torch
import gc
import time
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig, BitsAndBytesConfig
from awq import AutoAWQForCausalLM
import warnings
warnings.filterwarnings('ignore')

# GPU 사용 가능 확인
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 🔧 2. 유틸리티 함수들

In [ ]:
def get_model_size(model):
    """모델 크기를 MB 단위로 계산"""
    param_size = 0
    buffer_size = 0

    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_mb = (param_size + buffer_size) / 1024 / 1024
    return size_mb

def get_gpu_memory():
    """현재 GPU 메모리 사용량 확인"""
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / 1024**2  # MB
    return 0

def cleanup():
    """메모리 정리"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def test_inference(model, tokenizer, prompt="Hello, how are you?"):
    """간단한 추론 테스트"""
    inputs = tokenizer(prompt, return_tensors="pt")

    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}

    start_time = time.time()
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=20, do_sample=False)
    end_time = time.time()

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return {
        "text": generated_text,
        "time": end_time - start_time
    }

## 🎯 3. 테스트할 모델 선택

작은 모델을 사용하여 메모리 효율적으로 테스트합니다.

In [ ]:
# 테스트용 모델 (작은 모델 사용)
model_id = "facebook/opt-125m"  # 125M 파라미터 모델
# model_id = "facebook/opt-350m"  # 더 큰 모델 테스트시
# model_id = "facebook/opt-1.3b"  # 1.3B 모델 (GPU 메모리 충분시)

print(f"Selected Model: {model_id}")

## 📊 4. 기준 모델 (FP32/FP16)

In [ ]:
print("📊 Loading Base Model...")
cleanup()

tokenizer = AutoTokenizer.from_pretrained(model_id)

# FP16 모델 로드 (메모리 효율)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

base_size = get_model_size(base_model)
base_memory = get_gpu_memory()
base_result = test_inference(base_model, tokenizer)

print(f"✅ Base Model (FP16):")
print(f"   - Size: {base_size:.2f} MB")
print(f"   - GPU Memory: {base_memory:.2f} MB")
print(f"   - Inference Time: {base_result['time']:.3f} seconds")
print(f"   - Output: {base_result['text'][:100]}...")

# 메모리 정리
del base_model
cleanup()

## 🔵 5. BitsAndBytes 양자화 (8-bit & 4-bit)

In [ ]:
# 8-bit 양자화
print("🔵 Loading 8-bit Quantized Model...")
cleanup()

bnb_config_8bit = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16
)

model_8bit = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config_8bit,
    device_map="auto"
)

bit8_size = get_model_size(model_8bit)
bit8_memory = get_gpu_memory()
bit8_result = test_inference(model_8bit, tokenizer)

print(f"✅ 8-bit Quantized Model:")
print(f"   - Size: {bit8_size:.2f} MB")
print(f"   - Size Reduction: {(1 - bit8_size/base_size)*100:.1f}%")
print(f"   - GPU Memory: {bit8_memory:.2f} MB")
print(f"   - Inference Time: {bit8_result['time']:.3f} seconds")

del model_8bit
cleanup()

In [ ]:
# 4-bit 양자화
print("🔵 Loading 4-bit Quantized Model...")
cleanup()

bnb_config_4bit = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model_4bit = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config_4bit,
    device_map="auto"
)

bit4_size = get_model_size(model_4bit)
bit4_memory = get_gpu_memory()
bit4_result = test_inference(model_4bit, tokenizer)

print(f"✅ 4-bit Quantized Model:")
print(f"   - Size: {bit4_size:.2f} MB")
print(f"   - Size Reduction: {(1 - bit4_size/base_size)*100:.1f}%")
print(f"   - GPU Memory: {bit4_memory:.2f} MB")
print(f"   - Inference Time: {bit4_result['time']:.3f} seconds")

del model_4bit
cleanup()

## 🟢 6. GPTQ 양자화

In [ ]:
print("🟢 Loading GPTQ Quantized Model...")
cleanup()

# GPTQ 설정
gptq_config = GPTQConfig(
    bits=4,
    dataset="c4",  # 또는 커스텀 데이터셋 사용
    tokenizer=tokenizer
)

# 실제 양자화 수행 (시간이 걸립니다)
print("   Quantizing model (this may take a while)...")
model_gptq = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=gptq_config
)

gptq_size = get_model_size(model_gptq)
gptq_memory = get_gpu_memory()
gptq_result = test_inference(model_gptq, tokenizer)

print(f"✅ GPTQ Quantized Model:")
print(f"   - Size: {gptq_size:.2f} MB")
print(f"   - Size Reduction: {(1 - gptq_size/base_size)*100:.1f}%")
print(f"   - GPU Memory: {gptq_memory:.2f} MB")
print(f"   - Inference Time: {gptq_result['time']:.3f} seconds")

del model_gptq
cleanup()

## 🟡 7. AWQ 양자화

In [ ]:
print("🟡 Loading AWQ Quantized Model...")
cleanup()

# AWQ 모델 로드
# 주의: AWQ는 사전 양자화된 모델을 사용하거나 직접 양자화해야 합니다
try:
    # AWQ 모델 경로 (사전 양자화된 모델 사용)
    awq_model_path = model_id  # 또는 AWQ 양자화된 모델 경로

    # AWQ 양자화 설정
    quant_config = {
        "zero_point": True,
        "q_group_size": 128,
        "w_bit": 4,
        "version": "GEMM"
    }

    # 모델 로드
    model_awq = AutoAWQForCausalLM.from_pretrained(
        awq_model_path,
        safetensors=False
    )

    # 양자화 수행 (필요한 경우)
    print("   Quantizing with AWQ...")
    model_awq.quantize(
        tokenizer,
        quant_config=quant_config
    )

    awq_size = get_model_size(model_awq.model)
    awq_memory = get_gpu_memory()

    print(f"✅ AWQ Quantized Model:")
    print(f"   - Size: {awq_size:.2f} MB")
    print(f"   - Size Reduction: {(1 - awq_size/base_size)*100:.1f}%")
    print(f"   - GPU Memory: {awq_memory:.2f} MB")

    del model_awq

except Exception as e:
    print(f"⚠️ AWQ 양자화 실패: {e}")
    print("   AWQ는 특정 모델에만 지원되거나 사전 양자화된 모델이 필요할 수 있습니다.")

cleanup()

## 🔴 8. Dynamic Quantization (PyTorch Native)

In [ ]:
print("🔴 Applying Dynamic Quantization...")
cleanup()

# 기본 모델 다시 로드
base_model_cpu = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32
).to('cpu')  # Dynamic quantization은 CPU에서 동작

# Dynamic Quantization 적용
quantized_model = torch.quantization.quantize_dynamic(
    base_model_cpu,
    {torch.nn.Linear},  # Linear 레이어만 양자화
    dtype=torch.qint8
)

dynamic_size = get_model_size(quantized_model)

print(f"✅ Dynamic Quantized Model:")
print(f"   - Size: {dynamic_size:.2f} MB")
print(f"   - Size Reduction: {(1 - dynamic_size/base_size)*100:.1f}%")
print(f"   - Note: Dynamic quantization runs on CPU")

del base_model_cpu, quantized_model
cleanup()

## 📈 9. 결과 비교 및 요약

In [ ]:
import pandas as pd

# 결과 데이터 (실제 실행 결과로 업데이트 필요)
results = {
    "Method": ["Base (FP16)", "BitsAndBytes 8-bit", "BitsAndBytes 4-bit", "GPTQ 4-bit"],
    "Model Size (MB)": [base_size, bit8_size, bit4_size, gptq_size],
    "Size Reduction (%)": [0,
                           (1 - bit8_size/base_size)*100,
                           (1 - bit4_size/base_size)*100,
                           (1 - gptq_size/base_size)*100],
    "GPU Memory (MB)": [base_memory, bit8_memory, bit4_memory, gptq_memory],
    "Inference Time (s)": [base_result['time'],
                          bit8_result['time'],
                          bit4_result['time'],
                          gptq_result['time']]
}

df = pd.DataFrame(results)
df = df.round(2)

print("\n📊 === Quantization Methods Comparison ===")
print(df.to_string(index=False))

# 시각화
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Model Size
axes[0, 0].bar(df['Method'], df['Model Size (MB)'], color=['gray', 'blue', 'green', 'orange'])
axes[0, 0].set_title('Model Size Comparison')
axes[0, 0].set_ylabel('Size (MB)')
axes[0, 0].tick_params(axis='x', rotation=45)

# Size Reduction
axes[0, 1].bar(df['Method'], df['Size Reduction (%)'], color=['gray', 'blue', 'green', 'orange'])
axes[0, 1].set_title('Size Reduction')
axes[0, 1].set_ylabel('Reduction (%)')
axes[0, 1].tick_params(axis='x', rotation=45)

# GPU Memory
axes[1, 0].bar(df['Method'], df['GPU Memory (MB)'], color=['gray', 'blue', 'green', 'orange'])
axes[1, 0].set_title('GPU Memory Usage')
axes[1, 0].set_ylabel('Memory (MB)')
axes[1, 0].tick_params(axis='x', rotation=45)

# Inference Time
axes[1, 1].bar(df['Method'], df['Inference Time (s)'], color=['gray', 'blue', 'green', 'orange'])
axes[1, 1].set_title('Inference Time')
axes[1, 1].set_ylabel('Time (seconds)')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 💡 10. 양자화 방법별 특징 및 사용 가이드

### 📌 **BitsAndBytes**
- **장점**: 설정이 간단하고 즉시 사용 가능
- **단점**: 추론 속도가 상대적으로 느림
- **사용 시나리오**: 빠른 프로토타이핑, 메모리 제약이 있는 환경

### 📌 **GPTQ**
- **장점**: 우수한 압축률과 품질 유지
- **단점**: 양자화 과정이 오래 걸림 (캘리브레이션 필요)
- **사용 시나리오**: 프로덕션 배포, 품질이 중요한 경우

### 📌 **AWQ**
- **장점**: Activation을 고려한 양자화로 품질 우수
- **단점**: 모든 모델을 지원하지 않음
- **사용 시나리오**: 지원되는 모델에서 최적 성능 필요시

### 📌 **Dynamic Quantization**
- **장점**: 추가 라이브러리 불필요
- **단점**: GPU 지원 제한적
- **사용 시나리오**: CPU 추론, 간단한 양자화 필요시

## 🎯 11. 실전 팁

1. **모델 크기별 추천 양자화**:
   - 1B 이하: BitsAndBytes 4-bit
   - 1B-7B: GPTQ 또는 AWQ
   - 7B 이상: AWQ (가능한 경우) 또는 GPTQ

2. **하드웨어별 추천**:
   - Consumer GPU (8GB 이하): 4-bit 양자화 필수
   - Mid-range GPU (16GB): 8-bit 또는 4-bit
   - High-end GPU (24GB+): FP16 또는 8-bit

3. **품질 vs 속도 트레이드오프**:
   - 최고 품질: FP16 > 8-bit > GPTQ/AWQ > BitsAndBytes 4-bit
   - 최고 속도: AWQ > GPTQ > BitsAndBytes
   - 최소 메모리: 4-bit 양자화

In [ ]:
# 최종 메모리 정리
cleanup()
print("\n✅ 모든 테스트 완료! 메모리가 정리되었습니다.")